Using multiple SVMs

In [4]:
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.inception_v3 import preprocess_input
import tensorflow as tf

# Load your pre-trained model (replace 'model.h5' with the path to your HDF5 model)
model = tf.keras.models.load_model("AlexNetModel.hdf5")

def extract_features(image_paths):
    features = []
    for image_path in image_paths:
        img = cv2.imread(image_path)
        img = cv2.resize(img, (224, 224))  # Adjust input size as per the model's requirements
        img = preprocess_input(img)  # Preprocess image data as required by InceptionV3
        img = np.expand_dims(img, axis=0)
        feature_vector = model.predict(img)
        features.append(feature_vector)
    return np.vstack(features)

# Replace 'train_data' and 'valid_data' with the paths to your training and validation data directories
train_data_dir = 'data/train/'
valid_data_dir = 'data/valid/'

# List all image file paths in the training and validation directories
train_image_paths = [os.path.join(train_data_dir, filename) for filename in os.listdir(train_data_dir)]
valid_image_paths = [os.path.join(valid_data_dir, filename) for filename in os.listdir(valid_data_dir)]

# Extract feature vectors from the images using the pre-trained model
train_features = extract_features(train_image_paths)
valid_features = extract_features(valid_image_paths)

# Load labels for the training and validation data
# Assuming you have a list of labels for your images
labels = ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___healthy', 'Corn_(maize)___Northern_Leaf_Blight', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___healthy', 'Strawberry___Leaf_scorch', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_mosaic_virus', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus']  # List of labels for both training and validation images

# Train separate SVMs for each class using a one-vs-all strategy
svm_models = {}
for class_label in set(labels):
    # Create a binary label for the current class
    y_train_binary = np.array(labels) == class_label
    
    # Create an SVM classifier and train it
    svm = SVC(probability=True)
    svm.fit(train_features, y_train_binary)
    
    # Store the trained SVM model
    svm_models[class_label] = svm

# Classify a new image
def classify_image(image_path):
    feature_vector = extract_features([image_path])
    class_probabilities = {}
    for class_label, svm in svm_models.items():
        probability = svm.predict_proba(feature_vector)[0][1]
        class_probabilities[class_label] = probability
    predicted_class = max(class_probabilities, key=class_probabilities.get)
    return predicted_class

# Test the classification on a sample image (replace with your own image path)
sample_image_path = 'path/to/your/sample/image.jpg'
predicted_class = classify_image(sample_image_path)
print(f'Predicted class: {predicted_class}')


ImportError: Filepath looks like a hdf5 file but h5py is not available. filepath=AlexNetModel.hdf5